In [1]:
import os
if os.getcwd()[-16:] != "symbolicdefaults":
    os.chdir('../../symbolicdefaults')
import sys; sys.path.append('src')
import joblib
import pandas as pd

In [55]:
import numpy as np
from src import problem

In [ ]:
# Problem Statement:
We fix xgboosts "nrounds" parameter to 500.0.
This might possibly be too low, many people say this is the most important hyperparameter to tune.
We test this hypothesis using the following 2 experiments.

Simplification: 
- The best performance obtained on a given task (after normalization) is 1.0
- We choose the best performing configuration for nrounds between 400 and 600 in from the metaadata
- Compute the difference

On surrogates:
- Get best config per task from metadata and evaluate on surrogates. Take this performance as the best obtainable. 
- Use some optimizer on each task to find the best performing config given a fixed nrounds = 500
- Compute the difference

In [5]:
prob = problem.Problem("mlr_xgboost")
prob.data

,target,gamma,num.impute.selected.cpo,nrounds,eta,lambda,alpha,subsample,max_depth,min_child_weight,colsample_bytree,colsample_bylevel,booster,rate_drop,skip_drop,task_id
3723,-0.017298,1.535720e-01,impute.median,223.0,0.001590,0.228843,62.084000,0.693482,13.0,6.707660,0.978749,0.473018,gbtree,NaN,NaN,3893.0
3724,-0.061549,9.401830e-03,impute.hist,755.0,0.001127,0.764123,1.721730,0.814412,2.0,5.675130,0.040323,0.618070,gbtree,NaN,NaN,3893.0
3725,-0.005210,2.864510e-04,impute.mean,834.0,0.065828,0.700215,0.264038,0.554631,1.0,2.726210,0.929494,0.803628,gbtree,NaN,NaN,3893.0
3726,-0.009586,3.610640e-03,impute.mean,228.0,0.001555,0.003430,0.647020,0.657493,8.0,5.411290,0.540108,0.697829,gbtree,NaN,NaN,3893.0
3727,-0.033900,6.970520e-08,impute.hist,475.0,0.011307,388.926000,0.011893,0.286056,14.0,3.622780,0.990067,0.139293,gbtree,NaN,NaN,3893.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955205,-0.516667,1.209110e-01,impute.median,322.0,0.031712,0.000607,154.144000,0.373654,4.0,18.241900,0.957462,0.775648,gbtree,NaN,NaN,14954.0
2955206,-0.301852,4.970730e-05,impute.median,944.0,0.002305,115.469000,0.062564,0.771090,10.0,2.100100,0.377573,0.741008,gbtree,NaN,NaN,14954.0
2955207,-0.314815,1.894830e-03,impute.median,15.0,0.105778,902.268000,0.189815,0.832217,2.0,5.728530,0.903251,0.872140,gbtree,NaN,NaN,14954.0
2955208,-0.259259,9.203130e-02,impute.median,439.0,0.002710,4.666810,0.401621,0.541534,4.0,0.323353,0.544261,0.442392,gbtree,NaN,NaN,14954.0


In [66]:
def normalize_scores(ys):
    """
    Scale target to [0,1] where 1 is best, 0 is worst.
    """
    y = ys.values
    if (max(y) - min(y)) == 0:
        return pd.Series(np.zeros(shape = len(y)), index = ys.index)
    return pd.Series((y - min(y)) / (max(y) - min(y)), index = ys.index)

df = prob.data
df['target'] = df.groupby('task_id')['target'].apply(normalize_scores)
best = df.loc[df.groupby('task_id')['target'].idxmax(), ["target", "task_id"] + prob.hyperparameters].set_index("task_id")
best

,target,nrounds,eta,lambda,gamma,alpha,subsample,max_depth,min_child_weight,colsample_bytree,colsample_bylevel
task_id,,,,,,,,,,,
3.0,1.0,204.0,0.143361,0.014185,0.000092,0.007178,0.623329,6.0,0.086288,0.941184,0.288991
6.0,1.0,1535.0,0.074777,1.356180,0.028286,0.007443,0.843572,13.0,1.914750,0.744032,0.439877
11.0,1.0,1130.0,0.567326,0.007314,0.001316,0.016837,0.789042,12.0,1.638470,0.413200,0.854372
12.0,1.0,2014.0,0.085789,4.243020,0.000278,0.000004,0.948218,4.0,0.364724,0.790905,0.126456
14.0,1.0,695.0,0.026398,0.004425,0.017072,0.011400,0.979875,6.0,1.982460,0.803645,0.470955
...,...,...,...,...,...,...,...,...,...,...,...
189924.0,1.0,224.0,0.059091,0.034990,4.152330,0.029830,0.889363,15.0,1.100290,0.633104,0.493137
189927.0,1.0,690.0,0.027570,0.002937,0.678984,0.003367,0.471706,7.0,10.659300,0.704241,0.763229
189928.0,1.0,714.0,0.040349,0.009408,0.001713,0.089513,0.976969,15.0,1.525540,0.788839,0.546461


In [67]:
# Nrounds between 400 and 600
n500 = df.loc[df[df.nrounds.between(400, 600)].groupby('task_id')['target'].idxmax(), ["target", "task_id"] + prob.hyperparameters].set_index("task_id")
comp = n500[["target"]].join(best[["target"]], lsuffix = "_400-600", rsuffix="_best")

0.008539068510325572

In [69]:
# Compute the difference 
print(-(- comp["target_best"] + comp["target_400-600"]).mean())
#The average difference is 0.0085, i.e. we loose almost one percent of the maximum accuracy obtained by fixing this hyperparameter

0.008539068510325572
